# Ch 15. End-to-end nodule analysis, and where to go next
---
1. 결절 후보 생성
    - 세그멘테이션: 2차원 단면에 대해 결절이 의심되면 단면을 쌓아 3차원 복셀 리스트를 만든다.
    - 그룹화: 결절 후보들로 그룹화한다.
    - 샘플 튜플 만들기: I,R,C를 생성해서 샘플 튜플을 만든다.
2. 결절과 악성 종양 분류
    - 결절 분류: 위 과정을 통해 얻은 샘플 튜플을 분류한다.
    - ROC/AUC 메트릭: 새로운 메트릭을 정의한다.
    - 악성 모델 미세 튜닝: 메트릭을 살펴보고 모델의 성능을 향상시키기 위해 미세튜닝을 진행한다.
3. 엔드투엔드 탐지
    - IRC: CT를 세그멘테이션하여 분류할 결절 후보 샘플을 얻는다.
    - 결절 여부 판단: 위 후보에 대해 결절 분류를 진행한다.
    - 악성 여부 판단: 위 분류를 거친 결과를 악성인지 판단한다.


![image](https://user-images.githubusercontent.com/76675506/191728461-065ee43b-d88b-4127-9d86-a1a743073a3f.png)



## 검증셋 독립
*분류 모델*을 위한 데이터 분리는 결절 리스트에 대해 진행했고 *세그멘테이션 모델*을 위한 데이터 분리는 CT 스캔 데이터에 대해 이뤄졌다. 이는 곧 세그멘테이션의 검증 데이터가 분류 모델의 트레인 셋에 들어갈 수 있다는 의미이다.

`LunaDataset`을 `Luna2dSegmentationDataset`에 가져와서 미리 분리를 구현했다. 이를 통해 모델을 돌려본 결과 90 에포크에서 최대 F1 점수에 도달한다. 정확도를 다소 손해보더라도 이 결과에 만족한다.


## CT 세그멘테이션과 결절 후보 분류 연결
---
이제 그룹화를 진행한다. 세그멘테이션된 부분에서 중심 부분을 찾아 이를 IRC 형태의 샘플 튜플로 만든다.

CT를 도는 루프 부분을 제외하면 이때까지 살펴본 것과 비슷하다. CT를 도는 각 루프에서 모든 단면에 대해 세그멘테이션한 결과를 그룹화의 입력으로 사용한다. 그룹화 결과는 결절 분류기로 들어간 다음엔 악성 분류기로 들어가게 된다.

In [ ]:
# nodule_analysis.py:324, NoduleAnalysisApp.main
        for _, series_uid in series_iter: # 시리즈 uid 루프
            ct = getCt(series_uid) # CT를 얻는다
            mask_a = self.segmentCt(ct, series_uid) # 세그멘테이션

            candidateInfo_list = self.groupSegmentationOutput( # 표시된 복셀 그룹화
                series_uid, ct, mask_a)
            classifications_list = self.classifyCandidates( # 결절 분류기 실행
                ct, candidateInfo_list)

#### 세그멘테이션
CT 단면에 대해 세그멘테이션을 진행한다. 각 단면별로 받을 수 있도록 `series_uid`를 읽고 `__getitem__`을 호출한다.

또 출력 측면에서도 다르다. 이전에는 출력이 각 픽셀에 대해 결절이 맞는지 확률값으로 나타냈다. 이젠 이 예측값을 마스킹 배열을 통해 이진 분류한다. 경계값은 0.5로 진행한다.

`scipy.ndimage.morphology`을 이용해서 클린업을 추가로 할 수 있다. 이는 복셀을 지우고 내부만 유지하는 방식으로, 플래그된 영역을 더 작게 만든다. 이를 전부 구현하면 아래 코드와 같다.

In [ ]:
# nodule_analysis.py:384, .segmentCt

    def segmentCt(self, ct, series_uid):
        with torch.no_grad(): # 기울기는 필요없다.

            output_a = np.zeros_like(ct.hu_a, dtype=np.float32) # 확률 값이 담긴다.
            seg_dl = self.initSegmentationDl(series_uid)  #  배치에서 CT를 루프하는 데이터 로더
            for input_t, _, _, slice_ndx_list in seg_dl:

                input_g = input_t.to(self.device)
                prediction_g = self.seg_model(input_g)

                for i, slice_ndx in enumerate(slice_ndx_list):
                    output_a[slice_ndx] = prediction_g[i].cpu().numpy()

            mask_a = output_a > 0.5 # 이진 분류
            mask_a = morphology.binary_erosion(mask_a, iterations=1)

        return mask_a

#### 복셀을 그룹화해서 결절 후보 만들기
그룹화한 복셀을 분류기로 넘기기 위해 **컴포넌트 연결**(**Connected-component)** 알고리즘을 사용한다. 이는 연결되는 컴포넌트를 레이블하는 방식으로 `scipy.ndimage.measurements.label`을 사용한다. 이를 이용하면 동일한 경계선에 있는 픽셀을 동일한 그룹으로 묶어준다.

현재 세그멘테이션의 출력에 서로 인접하는 덩어리가 포함되어 있기 때문에 이는 효과적이다.

`scipy.ndimage.measurements.center_off_mass`를 이용해 중심값을 구해줄 수도 있다.

In [ ]:
# nodule_analysis.py:401
    def groupSegmentationOutput(self, series_uid,  ct, clean_a):
        candidateLabel_a, candidate_count = measurements.label(clean_a) # 복셀 레이블링
        centerIrc_list = measurements.center_of_mass( # 각 그룹의 중심값
            ct.hu_a.clip(-1000, 1000) + 1001,
            labels=candidateLabel_a,
            index=np.arange(1, candidate_count+1),
        )

        candidateInfo_list = []
        for i, center_irc in enumerate(centerIrc_list):
            center_xyz = irc2xyz(
                center_irc,
                ct.origin_xyz,
                ct.vxSize_xyz,
                ct.direction_a,
            )
            assert np.all(np.isfinite(center_irc)), repr(['irc', center_irc, i, candidate_count])
            assert np.all(np.isfinite(center_xyz)), repr(['xyz', center_xyz])
            candidateInfo_tup = \
                CandidateInfoTuple(False, False, False, 0.0, series_uid, center_xyz)
            candidateInfo_list.append(candidateInfo_tup)

        return candidateInfo_list

### 거짓 양성을 줄이기 위한 분류
---
아래 그림은 과정을 거치면서 버려지는 데이터를 표현한다. X로 표시된 부분이 버려지는 데이터 양이다.
1. 세그멘테이션: 세그멘테이션은 전체 CT 데이터로 시작한다. 세그멘테이션을 통해 약 97%의 데이터를 날린다.
2. 그룹화: 실제로 데이터를 버리진 않지만, 각각의 복셀을 결절 후보로 병합하는 과정에서 고려할 항목을 줄여준다.
3. 결절 분류: 결절 분류를 통해 데이터가 더 줄어든다.
4. 악성 분류


![image](https://user-images.githubusercontent.com/76675506/191732805-11bcb5d1-e372-4ceb-96cc-d45c6c9ac60c.png)

In [ ]:
# Listing 14.5 nodule_analysis.py:357, .classifyCandidates
    def classifyCandidates(self, ct, candidateInfo_list):
        cls_dl = self.initClassificationDl(candidateInfo_list)
        classifications_list = []
        for batch_ndx, batch_tup in enumerate(cls_dl):
            input_t, _, _, series_list, center_list = batch_tup

            input_g = input_t.to(self.device)
            with torch.no_grad():
                _, probability_nodule_g = self.cls_model(input_g) # 결절 분류기에 넣는다.
                if self.malignancy_model is not None: # 악성 분류 모델이 있다면 거기에도 넣는다.
                    _, probability_mal_g = self.malignancy_model(input_g)
                else:
                    probability_mal_g = torch.zeros_like(probability_nodule_g)

            zip_iter = zip(center_list,
                probability_nodule_g[:,1].tolist(),
                probability_mal_g[:,1].tolist())
            for center_irc, prob_nodule, prob_mal in zip_iter: # 계산 결과 리스트
                center_xyz = irc2xyz(center_irc,
                    direction_a=ct.direction_a,
                    origin_xyz=ct.origin_xyz,
                    vxSize_xyz=ct.vxSize_xyz,
                )
                cls_tup = (prob_nodule, prob_mal, center_xyz, center_irc)
                classifications_list.append(cls_tup)
        return classifications_list


위 코드를 토대로 얻은 confusion matrix를 살펴보자.

![image](https://user-images.githubusercontent.com/76675506/191734817-1d022685-d5c5-4113-97d0-9b4751ef53df.png)

`Complete Miss` 열은 세그멘테이션 모델이 결절에 아무 표시도 하지 않은 경우다. 세그멘테이션 모델을 만들 때 높은 재현율을 의식해서 만들었기 때문에 결절이 아닌 경우가 많지만, 결절 분류기가 이를 잘 걸러냈다.


## 정량적인 검증
---
이제 전체 데이터셋에 대해 검증을 시행해본다.


![image](https://user-images.githubusercontent.com/76675506/191735511-e88ed9bb-8a8f-4c30-a1bd-88e88d1b4eab.png)


154개의 결절에서 132개, 즉 85%를 탐지했다. 놓친 22개 중 13개는 세그멘테이션에서 놓쳤기 때문에 이를 개선해야 한다.

## 악성 예측

현재 모델로 양성과 악성을 구분할 수 있을까? 우리가 살펴본 LUNA 데이터는 결절 탐지에만 집중한 대회이기 때문에 LIDC 데이터를 이용한다. LIDC엔 악성 수준 정보가 포함되어 있다. 파이썬에서는 pyLIDC`라이브러리를 설치할 수 있다.


LIDC의 악성 수준 정보를 사용하기 위해 LIDC의 애노테이션 정보를 LUNA의 cadidate의 좌표정보와 연결해야 한다. LIDC에는 결절 별로 악성일 가능성 1~5가 인코딩되어있다. 여기서는 4이상이면 악성으로 간주한다.


데이터를 합치는 방법은 10장과 동일하므로 생략한다.

### AUC 베이스라인: 직경으로 분류

결절의 악성 여부를 예측하기 위해 분류기에 결절의 직경을 유일한 입력으로 사용한다면 어떻게 될까? 좋은 분류기가 될 수는 없겠지만 경계값을 기준으로 악성을 분류하는 방법은 꽤 괜찮다고 알려져 있다. 여기서 적당한 경계값을 찾는게 관건이다.

12장의 재현율을 생각해보면 경계값을 어떻게 정하느냐에 따라 참 양성, 거짓 양성, 참 음성, 거짓 음성이 바뀐다. 결절을 악성으로 예측하는 경계값을 낮추면 참 양성을 올릴 수 있지만 거짓 양성 수도 올라간다.


경계값의 범위에 대해 생각해보자. 모든 샘플이 양성으로 분류되는 경우가 최솟값이고 반대로 모든 샘플이 음성으로 분류되는 경우는 최대값이다. 극단적인 경우는 각각 악성이 하나도 없어서 거짓 양성 비율(FPR)과 참 양성 비율(TPR)이 0이 되는 경우와 모든 샘플이 악성이라 참 음성과 거짓 음성이 없어서 FPR과 TPR이 1인 경우가 있다.


우리 데이터에서는 결절은 [3.25, 22.78]의 범위를 가진다. 이 두 값 사이에서 경계값을 고르면 FRP, TPR 경계값을 계산할 수 있다. 아래 그래프는 FPR을 X 축으로 TPR을 Y 축으로 나타낸 결과다. 이러한 그래프를 **ROC**(**Receiver Operating Characteristic**)이라고 부른다. 색칠된 영역은 **AUC**(**Area Under Curve**)라고 부른다. AUC는 0에서 1사이의 값을 가지며 높을 수록 좋다.


![image](https://user-images.githubusercontent.com/76675506/191741283-5d84c401-45cc-41de-a9fb-97826fa20256.png)


위 그래프에서 표시된 두 경계값을 살펴보자. 5.42 보다 작다면 TPR만 떨어뜨린다. 반대로 10.55보다 크면 아무런 이득 없이 악성 결절을 양성으로 분류하게 된다. 따라서 최적의 값은 이 두 값 사이에 존재한다.


이 값을 계산하기 위해 결절로 애노테이션된 데이터는 날리고 악성 레이블과 직경을 얻는다.

In [ ]:
# Listing 14.8 p2ch14_malben_baseline.ipynb
ds = p2ch14.dsets.MalignantLunaDataset(val_stride=10, isValSet_bool=True)  # 데이터셋에서 양성과 악성 결절 리스트를 얻는다.
nodules = ds.ben_list + ds.mal_list
is_mal = torch.tensor([n.isMal_bool for n in nodules])  # 악성 상태와 직경 리스트를 얻는다.
diam  = torch.tensor([n.diameter_mm for n in nodules])
num_mal = is_mal.sum()  # TPR과 FPR 정규화를 위해 악성과 양성 결절 수를 얻는다.
num_ben = len(is_mal) - num_mal

ROC 곡선을 계산하기 위해 경계가 될 값이 필요하다. 이를 위해 두 경계값을 입력받는 `torch.linspace`를 이용한다.

In [ ]:
threshold = torch.linspace(diam.max(), diam.min()) # 주어진 값에서 최소, 최대 값을 사용한다.

행은 경계를 나타내고 열은 샘플별 정보를 의미하는 2차원 텐서를 만든다. 각 원소의 값은 각 샘플이 양성으로 예측되었는지를 불리언 값으로 나타낸다. 이 불리언 텐서를 통해서 샘플이 양성인지 악성인지 구별한다. True 수를 세서 악성 결절의 수로 나누면 TPR, 양성 결절의 수로 나누면 FPR을 얻을 수 있다. 이를 이용해 ROC 곡선을 만든다.

In [ ]:
predictions = (diam[None] >= threshold[:, None])  # <1>
tp_diam = (predictions & is_mal[None]).sum(1).float() / num_mal  # <2>
fp_diam = (predictions & ~is_mal[None]).sum(1).float() / num_ben

AUC의 면적을 계산하기 위해서는 수치 적분을한다.

In [ ]:
fp_diam_diff =  fp_diam[1:] - fp_diam[:-1]
tp_diam_avg  = (tp_diam[1:] + tp_diam[:-1])/2
auc_diam = (fp_diam_diff * tp_diam_avg).sum()

### 이전 가중치의 재활용: 미세 튜닝

결과를 빨리 얻는 방법 중 하나는 이미 훈련된 모델을 사용하는 것이다. 이를 **미세 튜닝(fine tuning)** 혹은 **전이 학습**(**Transfer learning**)이라고 한다. 이 때 중간 과정에서 나온 값을 이미지에서 추출된 피처라고 볼 수 있다. 이런 방식은 일반적으로 잘 동작한다.


비슷한 데이터에서 훈련된 결절 분류 신경망이 있으므로 이를 사용한다. 가장 간단한 미세튜닝 방법으로 랜덤 초기화는 그대로 두고 `head_linear` 부분을 잘라낸다. 먼저 이렇게 시도해본 후, `head_linear`와 마지막 컨볼루션 블럭을 재훈련 시키면서 다른 방법도 고민해보자. 이때 아래 사항을 진행해야 한다.

- 시작할 때 사용할 모델의 가중치를 마지막 선형 계층을 제외하고 로딩한다.
- 훈련하지 않을 부분의 파라미터에 영향을 미칠 기울기를 차단한다.


![image](https://user-images.githubusercontent.com/76675506/191946360-e6090ec0-9002-4300-b124-61be4569af7d.png)



In [ ]:
# Listing 14.9 training.py:124, .initModel

        if self.cli_args.finetune:
            d = torch.load(self.cli_args.finetune, map_location='cpu')
            model_blocks = [
                n for n, subm in model.named_children()
                if len(list(subm.parameters())) > 0 # 파라미터를 가진 최상위 모듈은 제외한다.
            ]
            finetune_blocks = model_blocks[-self.cli_args.finetune_depth:] # 마지막 finetune_depth 블럭을 얻는다. 미세 튜닝 시 기본값은 1이다.
            log.info(f"finetuning from {self.cli_args.finetune}, blocks {' '.join(finetune_blocks)}")
            model.load_state_dict(
                {
                    k: v for k,v in d['model_state'].items()
                    if k.split('.')[0] not in model_blocks[-1] # 마지막 블럭을 제외시켜 로딩하지 않는다. 완전히 초기화된 모델에서 시작하면 앞단의 분류기 출력이 '결절'을 의미하므로 모든 결절을 악성으로 판단한다.
                },
                strict=False, # 모듈의 가중치를 로딩한다.
            )
            for n, p in model.named_parameters():
                if n.split('.')[0] not in finetune_blocks: # finetune_blocks가 아닌 경우는 기울기가 필요 없다.
                    p.requires_grad_(False)

검증셋에 대해 모델을 돌려서 ROC 커브를 그려보면 아래 그림과 같다. 랜덤인 경우보다는 좋지만 기준치를 넘지 못하고 있기 때문에 살펴볼 필요가 있다.


![image](https://user-images.githubusercontent.com/76675506/191947410-2eccab85-c14e-42db-8751-6dfe820d85d4.png)

아래 그림은 훈련에 대한 텐서보드 그래프이다. 검증 손실을 보면 AUC가 천천히 증가하면서 손실값이 줄어들고 훈련 손실은 0으로 가는 대신 높은 지점에서 능선을 만든다. 나쁘지 않은 결과지만 손실값 변화에 정체가 있는 이유를 생각해봐야 한다.

- 결절과 비결절을 분류하는 신경망 훈련에서 얻은 피처가 악성 탐지에는 도움이 되지 않는 경우
- 헤드(유일하게 훈련하고 있는 부분)의 capacity가 충분하지 않은 경우
- 신경망의 capacity가 전반적으로 작은 경우

![image](https://user-images.githubusercontent.com/76675506/191947865-f56f055e-ab7b-45e2-9081-54af43411488.png)

미세 튜닝 단계에서 완전 연결 훈련으로 충분하지 않았다면 컨볼루션 블럭까지 포함해볼 수 있다. `block4`를 포함해서 진행한다. 그 결과 거짓 양성이 거의 없이 75%의 악성 결절을 찾아냈다. 직경 기준치가 주는 65%보다는 훨씬 낫다.

![image](https://user-images.githubusercontent.com/76675506/191948374-c0dd9fc1-07c6-4916-88d4-c2d9ded93248.png)

손실값 곡선을 보면 모델이 매우 일찍 과적합하는 것을 알 수 있다. 이를 해결하기 위해 정규화 메소드를 점검해야 한다.

![image](https://user-images.githubusercontent.com/76675506/191948661-e2b7cf33-40b4-4514-ad4e-979fb9298e69.png)

미세 튜닝을 하는 다른 방법으로
- 제일 위에서부터 고정되어 있던 계층 풀기
- 뒷부분 계층은 일반적인 학습률로 훈련시키고 낮은 계층에서는 작은 학습률로 훈련
등이 있다.

### 히스토그램

악성을 예측한 확률값을 히스토그램으로 나타낼 수 있다. 실측값을 기준으로 양성과 악성에 대한 두 개의 히스토그램을 만들어보자.

> 결과를 시각화할 때는 네이밍과 필요 없는 데이터 제거를 신경써야 한다.

In [ ]:
METRICS_LABEL_NDX=0 # 레이블 수
METRICS_PRED_NDX=1 # 데이터 수
METRICS_PRED_P_NDX=2 # 예측 확률을 담은 새 인덱스
METRICS_LOSS_NDX=3
METRICS_SIZE = 4 # 표시할 학습 샘플 수

bins = np.linspace(0, 1)

writer.add_histogram(
    'label_neg',
    metrics_t[METRICS_PRED_P_NDX, negLabel_mask],
    self.totalTrainingSamples_count,
    bins=bins
)
writer.add_histogram(
    'label_pos',
    metrics_t[METRICS_PRED_P_NDX, posLabel_mask],
    self.totalTrainingSamples_count,
    bins=bins
)

양성 샘플의 예측 분포가 에포크마다 어떻게 변하는지 살펴보자. 첫 줄의 비결절 양성인 경우는 신경망이 악성이 아님을 확신하는 의미의 산 모양을 볼 수 있다. 비슷하게 오른쪽에도 악성 샘플에 대한 산 모양이 보인다.


하지만 자세히 살펴보면 한 계층에서 미세 튜닝의 capacity 문제를 볼 수 있다. 좌상단의 히스토그램들에서는 덩어리가 그다지 줄어들지 않는 것을 볼 수 있다. 심지어 1.0 부근에서는 작게 솟은 지점도 보이고 전체 범위에 대해 확률 질량이 퍼져나가고 있다.


오른쪽 검증 결과를 살펴보면 '올바른' 영역에서 멀어진 확률 질량(probability mass)이 다이어그램 우하단의 악성 샘플보다 우상단의 악성이 아닌 샘플에 대해 훨씬 더 큰 듯이 보인다. 즉 이는 신경망이 악성이 아닌 샘플을 악성 샘플보다 자주 잘못 판단하는 것이다. 그러므로 데이터를 다시 밸런싱해서 악성이 아닌 샘플을 더 보여줄 필요가 있다.

![image](https://user-images.githubusercontent.com/76675506/191950602-98e4b0e5-9f75-4a16-9b1b-4f1bb7990247.png)

깊이2에 대한 결과는 아래와 같다. 훈련 부분은 정답에서 매우 뾰족한 봉우리를 보여주지만 다른 곳에서는 발견할 수 없다. 이는 훈련이 잘 동작한다는 의미이다.

검증에서 가장 두드러진 차이인 악성에 대한 예측 확률이 0인 지점의 작은 봉우리를 우하단 히스토그램에서 볼 수 있다. 따라서 현재 시스템적 문제는 <u>악성 샘플을 악성이 아닌 것으로 잘못 분류하는 점</u>이다. 위 상황과는 반대 상황이다.


![image](https://user-images.githubusercontent.com/76675506/191952242-3188794e-142d-4aa1-a5d6-b76b84308030.png)

텐서보드는 자체적인 ROC 커브 그래프를 지원하지 않기 때문에, `matplotlib`을 사용해서 그래프를 그린다. 그래프를 살펴보면 왼쪽의 깊이2 미세 튜닝이 과적합하는 것을 볼 수 있다. 반면 헤드 부분만 미세튜닝한 오른쪽은 과적합하지 않는다.

![image](https://user-images.githubusercontent.com/76675506/191952995-4ee6130a-9151-43c1-8a1e-b168bc7a32de.png)

### 진단을 통해 보는것

최종적으로 코드를 이어붙이고 결과를 살펴보자. 엔드투엔드로 결절 85%를 탐지하고 악성중에 70%를 검출했다. 많은 거짓 양성이 있지만 그래도 이 모델을 통해 분류할 일이 줄어들었다.

![image](https://user-images.githubusercontent.com/76675506/191953221-c8f5150c-e1f8-4694-823a-07f24e78b631.png)

검증셋으로 모델을 훈련시키지는 않았지만 검증셋에서의 모델 성능을 보고 제일 좋은 훈련 에포크를 선택한 방식에 문제가 있다. 일종의 데이터 누출에 해당하기 때문이다. 즉 이는 새로운 데이터에 대해서는 모델이 동일한 성능을 보여주지 못할 가능성이 높다는 이야기이다.

이러한 문제를 방지하기 위해 데이터셋을 검증셋으로 모델을 훈련시키지는 않았지만 검증셋에서의 모델 성능을 보고 제일 좋은 훈련 에포크를 선택한 방식에 문제가 있다. 일종의 데이터 누출에 해당하기 때문이다. 즉 이는 새로운 데이터에 대해서는 모델이 동일한 성능을 보여주지 못할 가능성이 높다는 이야기이다.

이러한 문제를 방지하기 위해 데이터셋을 **훈련셋, 검증셋, 테스트셋** 세 가지로 나누기도 한다.


## 다음 단계는?

### 과적합 방지: 더 나은 정규화

지금까지 만든 모델들 대부분에서 과적합이 발생했다. 이를 위해 데이터를 밸런싱하고 증강시키곤 했다. 또 검증셋에 대해 과적합이 시작되면 훈련을 중단시켜서 빠져나왔다. 먼저 과적합을 시키고 과적합을 줄이는 식의 패턴은 일반적인 절차다.


#### 전통적인 정규화와 증강 방법

- 드랍아웃
- 증강: Digital crumple, Elastic deformation

#### 더 추상적인 증강 방법

- **레이블 스무딩**(**label smoothing**): 원핫 분포 대신 '잘못된' 클래스에 작은 확률 질량을 배치
- **믹스업**(**mixup**): 손실값에 대한 선형성을 가정하고 랜덤하게 입력과 레이블을 동시에 채움. 예측 안정성을 높일 수 있다.

#### 앙상블링

- **앙상블링**(**ensembling**): 여러개의 모델을 훈련하고 평가한 뒤, 예측할 때는 모든 모델을 실행해서 예측을 평균하는 방식. 평균을 사용하는 방식은 **확률적 가중치 평균**(**Stochastic weight averaging**)의 핵심이다.

#### 신경망이 학습할 내용 일반화하기

- **멀티태스크 학습**(**Multitask learning**): 평가에 사용하는 출력 외에 다른 출력에 대해서도 모델이 학습하도록 만드는 방식. 예를 들어 결절과 비결절을 훈련시킬 때 동시에 양성과 악성도 구분한다.
- **준지도 학습**(**semi-supervised learning**): 비지도 데이터 증강(unsupervised data augmentation)에 최근 제안된 방식이다. 예측이 맞을지 모르지만 데이터 증강 여부와 상관없이 모델이 일관되게 예측하도록 만든다.

  1. 평소처럼 데이터로 모델을 훈련시킨다.
  2. 레이블 되지 않은 데이터에 대해 증강되지 않은 샘플을 대상으로 예측한다.
  3. 증강된 샘플을 대상으로 증강되지 않은 샘플과 동일한 예측이 가능하게끔 모델을 훈련한다.
- **프리텍스트 작업**(**pretext task)**:**자기 지도 학습**(**self-supervised learning**)의 일종으로 task를 만들어 성능을 향상시킨다. 대표적인 방식으로 일부러 입력을 손상시키는 방법이 잇다. 그 후 모델의 많은 부분을 공유하면서 원래 모양을 재구성하도록 훈련시키거나 분류기가 손상된 데이터와 실제 데이터를 구분할 수 있도록 훈련시킨다.
- **대조 학습**(**contrastive learninh**)
- **모멘텀 대조**(**Momentum contrast**)

### 훈련 데이터 개선

악성 분류 문제에서 여러 방사선 전문의가 세밀하게 분류한 데이터셋이라고 설명했었다. 악성인가 아닌가, 라는 이분법으로 나누면서 버려버린 데이터를 활용하는 방법은 어떤 것이 있을까? 가장 쉬운 방법은 다섯 개의 클래스를 그대로 이용하는 것이다. 이를 원 핫 인코딩하고 주어진 결절에 대한 레이블에 대해 평균을 내는 것이다. 예를 들어 4명의 방사선 전문의가 결절을 본 후 2명은 '불확실', 한명은 '다소 의심', 나머지 한명이 '악성 의심'이라고 분류했다면 모델의 출력과 벡터 (0, 0, 0.5, 0.25, 0.25)로 타겟의 확률 분포와의 크로스 엔트로피로 훈련할 수 있다.


세그멘테이션 측면에서는 우리가 만든 모델과 PyLIDC에서 제공하는 마스크를 서로 비교해볼 수 있다. LIDC는 여러 방사선 전문의가 애노테이션을 달았고 '높은 일치도' 그룹과 '낮은 일치도' 그룹으로 나눌 수 있다. 이를 분류기가 쉽게 분류해내면 '쉬움'에 넣고 그렇지 않으면 '어려움' 그룹에 넣어서 앞선 두 그룹과 비교하는 것도 흥미로울 것이다.


결절을 악성 유형별로도 나눠볼 수 있다. 전문가가 훈련 데이터를 살펴보고 암 유형에 따라 결절을 분류한 뒤, 모델에 해당 유형을 알려준다면 더 효과적인 훈련이 이루어질 수 있다.